In [ ]:
import geopandas as gpds
import networkx as nx
import time
import random
import matplotlib.pyplot as plt
import json

In [ ]:
roads = gpds.read_file('data/sfo_roads.shp')
nodes = gpds.read_file('data/sfo_nodes.shp')

In [ ]:
'''
Making shape to NetworkX shortest_path function input format
'''

Edges_Nx = []

for i in range(len(roads)):
    Edge_Nx = (roads.STARTID.iloc[i],roads.ENDID.iloc[i], {'weight':(roads.LENGTH.iloc[i])})
    Edges_Nx.append(Edge_Nx)
    
    
G = nx.Graph()
for i in range(len(roads)):
    G.add_node(i)
G.add_edges_from(Edges_Nx)

In [11]:
def Random_Dijkstra_NetworkX(OutputName):
    '''
    
    This function returns the nodes on the path with QGIS SQL query format.
    Also it generates the geojson output which contains the path nodes as point layer.
    
    '''
    Nodes = []

    # Random nodes selecting
    for i in range(len(nodes)):
        NodeId = str(int(nodes["ID"][i]))

        Nodes.append(NodeId)

    
    index_start = random.randint(0,len(nodes))
    index_finish = random.randint(0,len(nodes))

    Starting_Node = Nodes[index_start]
    Ending_Node = Nodes[index_finish]
    
    
    out_dict = {
    "type": "FeatureCollection",
    "name": OutputName,
    "crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:EPSG::3857" } },
    "features": [
    ]
    }

    
    try:
        NX_path = nx.shortest_path(G, source=int(Starting_Node), target=int(Ending_Node), weight='weight')
    
        NX_path_SQL = ""

        for j in range(len(NX_path)):
            query = (' "ID" = {} or '.format(NX_path[j]) )
    #           
            NX_path_SQL += query
        
        
        NX_path_SQL = NX_path_SQL[:-4]
        
        
        for i in range(len(NX_path)):
    
    

            X_coord = nodes[nodes["ID"] == NX_path[i]].geometry.item().x #["898840504"]
            Y_coord = nodes[nodes["ID"] == NX_path[i]].geometry.item().y



            point_feature = { "type": "Feature", "properties": { "ID": i }, "geometry": { "type": "Point", "coordinates": [ X_coord, Y_coord ] } }

            out_dict["features"].append(point_feature)
        
        
        OutputString = "{}.geojson".format(OutputName) 

        with open(OutputString, 'w') as file:
             file.write(json.dumps(out_dict)) # use `json.loads` to do the reverse

    except:
        NX_path_SQL = "There is no path between {} and {}.".format(Starting_Node,Ending_Node)
        
        
    return NX_path_SQL

    

In [15]:
alexxxx = Random_Dijkstra_NetworkX("path")

In [16]:
print(alexxxx)

 "ID" = 48524591 or  "ID" = 48524595 or  "ID" = 48524596 or  "ID" = 48524598 or  "ID" = 48524600 or  "ID" = 48530509 or  "ID" = 48530510 or  "ID" = 48530514 or  "ID" = 851319362 or  "ID" = 851319361 or  "ID" = 48530527 or  "ID" = 851319359 or  "ID" = 48530528 or  "ID" = 48530550 or  "ID" = 946460163 or  "ID" = 946460162 or  "ID" = 947213705 or  "ID" = 48530547 or  "ID" = 48530556 or  "ID" = 816720638 or  "ID" = 886118424 or  "ID" = 850784614 or  "ID" = 968054914 or  "ID" = 48530609 or  "ID" = 847345328 or  "ID" = 847345327 or  "ID" = 855080308 or  "ID" = 48530601 or  "ID" = 855080309 or  "ID" = 276604275 or  "ID" = 968081241 or  "ID" = 48530628 or  "ID" = 968081242 or  "ID" = 816720639 or  "ID" = 294167512 or  "ID" = 968058482 or  "ID" = 48530664 or  "ID" = 48530669 or  "ID" = 816720715 or  "ID" = 872523076 or  "ID" = 872483213 or  "ID" = 849382730 or  "ID" = 816720703 or  "ID" = 816720702 or  "ID" = 48530582 or  "ID" = 968053457 or  "ID" = 48530784 or  "ID" = 48530797 or  "ID" = 80172